In [1]:
import os
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [2]:
#change work directory to actual
os.chdir('./Documents/Python/Gidro')
os.getcwd()

'/Users/Documents/Python/Gidro'

In [3]:
#Load xlsx workbook
wb = load_workbook(filename = 'gidro.xlsx')

In [4]:
#Dictionary for translate column number to calumn letter.
#Next block ColNum.translate(t) will use it.

s = ''

intab = "0123456789"
outtab = "ABCDEFGHIJ"
none = ''

t = s.maketrans(intab, outtab, none)

In [5]:
#Import data from one xls sheet. sh parameter - sheet name: Lst1 .. Lst5

def create_xls_dataframe(sh):

    if sh.max_column > 10:
        return 'Columns indexes more than J are not supported :( '
        
    print ( 'Creating DataFrame with {} entry'.format(sh.max_row ))
    ds = pd.DataFrame()
    
    # Create DataSet named column indexes
    for i in range(sh.max_column):
        c = str(i).translate(t) + '1'
        ds[ sh[c].value] = None
    
    #Export each row of data
    for j in np.arange(2, sh.max_row + 1 ):
        
        if j % 1000 == 0:
            print ('{} rows already read. {} rows left'.format(j, sh.max_row - j))
        d = dict()
        
        #Consequentially read each column value of current row to dict
        for i in range(sh.max_column):
            c = str(i).translate(t) + '1'            
            d [ sh [ c ].value ] = sh [ str(i).translate(t) + str(j)].value 
        
    
        ds = ds.append (d , ignore_index=True)
        
            
    print ( 'DataFrame created. Shape is {}'.format(ds.shape))
   
    return ds


    

In [6]:
#Main data sheet. Export takes about 15 minutes.
s = wb['Lst1']
data = create_xls_dataframe(s)
del s

data.info()


Creating DataFrame with 35285 entry
1000 rows already read. 34285 rows left
2000 rows already read. 33285 rows left
3000 rows already read. 32285 rows left
4000 rows already read. 31285 rows left
5000 rows already read. 30285 rows left
6000 rows already read. 29285 rows left
7000 rows already read. 28285 rows left
8000 rows already read. 27285 rows left
9000 rows already read. 26285 rows left
10000 rows already read. 25285 rows left
11000 rows already read. 24285 rows left
12000 rows already read. 23285 rows left
13000 rows already read. 22285 rows left
14000 rows already read. 21285 rows left
15000 rows already read. 20285 rows left
16000 rows already read. 19285 rows left
17000 rows already read. 18285 rows left
18000 rows already read. 17285 rows left
19000 rows already read. 16285 rows left
20000 rows already read. 15285 rows left
21000 rows already read. 14285 rows left
22000 rows already read. 13285 rows left
23000 rows already read. 12285 rows left
24000 rows already read. 11285

In [7]:
#Criteria sheet
s = wb['Lst2']
crits = create_xls_dataframe(s)
del s


crits.info()

Creating DataFrame with 77 entry
DataFrame created. Shape is (76, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 4 columns):
Класс оборудования    76 non-null object
Критерий              76 non-null object
Вес                   76 non-null float64
Модель деградации     76 non-null object
dtypes: float64(1), object(3)
memory usage: 2.5+ KB


In [8]:
#Unit list
s = wb['Lst3']
units = create_xls_dataframe(s)
del s

units.info()

Creating DataFrame with 23 entry
DataFrame created. Shape is (22, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
Класс оборудования    22 non-null object
Узел                  22 non-null object
Вес                   22 non-null float64
dtypes: float64(1), object(2)
memory usage: 608.0+ bytes


In [9]:
#Models list
s = wb['Lst4']
models = create_xls_dataframe(s)
del s

models.info()

Creating DataFrame with 102 entry
DataFrame created. Shape is (101, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 10 columns):
Год         101 non-null object
Модель 1    101 non-null object
Модель 2    101 non-null object
Модель 3    101 non-null object
Модель 4    101 non-null object
Модель 5    101 non-null object
Модель 6    101 non-null object
Модель 7    101 non-null object
Модель 8    101 non-null object
Модель 9    101 non-null object
dtypes: object(10)
memory usage: 8.0+ KB


In [10]:
#Years subsheet. Transfered to separate sheet from models.
s = wb['Lst5']
years = create_xls_dataframe(s)
del s

years.info()

Creating DataFrame with 10 entry
DataFrame created. Shape is (9, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 5 columns):
Модель деградации    9 non-null object
3                    9 non-null object
2                    9 non-null object
1                    9 non-null object
0                    9 non-null object
dtypes: object(5)
memory usage: 440.0+ bytes


In [11]:
def degrade_model (index):

    a = data.iloc[index]
    a5 = a['Критерий']
    a7 = a['Класс оборудования']      
    a8 = a7 + a5
            
    degrade_model = crits[ (crits['Критерий'].values == a5) & (crits['Класс оборудования'].values == a7) ]

    
    
    # If found More or less than one model, report about it!
    if degrade_model.shape[0] != 1: 
        print('Row {} has problem. {} in critera directory has {} corresponds'.format(index, 
                                                                                      a8, degrade_model.shape[0]))
        return 'Model 0'
        
    return degrade_model['Модель деградации'].values[0]
        
            


In [12]:
def count_year(index):
    a = data.iloc[index]
    a6 = a['Оценка']    
    year_string  =  years [( years['Модель деградации'] == degrade_model(index) ) ] 
    
    return  year_string[a6] .values[0]


In [13]:
def crit_weight (index):

    a = data.iloc[index]
    a5 = a['Критерий']
    a7 = a['Класс оборудования']      
    a8 = a7 + a5
            
    degrade_model = crits[ (crits['Критерий'].values == a5) & (crits['Класс оборудования'].values == a7) ]
    
    # If found More or less than one model, report about it! 
    if degrade_model.shape[0] != 1: 
        print('Row {} has problem. {} in critera directory has {} corresponds'.format(index, 
                                                                                      a8, 
                                                                                      degrade_model.shape[0]))
        return 99999  #Extremly error value. Can be easily detected
        
    return float(degrade_model['Вес'].values[0])


        
            
    

In [14]:
def unit_weight(index):
    a = data.iloc[index]
    a4 = a['Узел']
    a5 = a['Критерий']
    a7 = a['Класс оборудования']      
    a8 = a7  + a5
            
    unit_model = units[ (units['Класс оборудования'].values == a7) & (units['Узел'].values == a4) ]
    
    if unit_model.shape[0] != 1: 
        print('Row {} has problem. {} in critera directory has {} corresponds'.format(index, 
                                                                                      a8, 
                                                                                      unit_model.shape[0]))
        # If found More or less than one model, report about it! 
        return 99999
        
    return float(unit_model['Вес'].values[0])


In [15]:
def prognosis_crit(index):
      
    m = degrade_model(index)
    
    return np.array( models[m].iloc[count_year(index)-1:count_year(index)+15] ) 
    


In [17]:
#Make list of all unique Units
unit_dict = dict()
    
for i in range(data.shape[0]):
    a = data.iloc[i]
    year = a['Год']
    branch = a['Филиал']
    unit = a['Единица оборудования']
   
    unit_dict[(year,branch,unit)] = None 

In [18]:
j = 0
for i in unit_dict:
    unit_dict[i] = j
    j += 1

In [95]:
def crits_for_unit(unit_name):
    b  = crits[crits['Класс оборудования'] == unit_name]
    
    return np.array(b['Критерий'])

print(crits_for_unit('Гидрогенератор').shape )
print(crits_for_unit('Турбина ПЛ').shape)
print(crits_for_unit('Турбина РО').shape)



(22,)
(27,)
(27,)


In [29]:
def new_line (index):
    a = data.iloc[index]
    a1 = a['Год']
    a2 = a['Филиал']
    a3 = a['Единица оборудования']
    a4 = a['Узел']
    a5 = a['Критерий']
    a6 = a['Оценка']
    a7 = a['Класс оборудования']  
    if ( a1,a2,a3 ) in unit_dict:
        a9 = unit_dict  [( a1,a2,a3 )] 
    else:
        a9 = 'Not in dict Year/Branch/Unit'
    prognosis = prognosis_crit(index)  * crit_weight(index) * unit_weight(index)
    a10 = prognosis [0]
    a11 = prognosis [1]
    a12 = prognosis [2]
    a13 = prognosis [3]
    a14 = prognosis [4]
    a15 = prognosis [5]
    a16 = prognosis [6]
    a17 = prognosis [7]
    a18 = prognosis [8]
    a19 = prognosis [9]
    a20 = prognosis [10]
    a21 = prognosis [11]
    a22 = prognosis [12]
    a23 = prognosis [13]
    a24 = prognosis [14]
    a25 = prognosis [15]
    
    d = {'Год':a1,
             'Филиал':a2,
             'Единица оборудования':a3,
             'Узел':a4,
             'Критерий':a5,
             'Оценка':a6,
             'Класс оборудования':a7,
             'Индекс ИТС': a9,
             'Прогноз0': prognosis [0],
             'Прогноз1': prognosis [1],
             'Прогноз2': prognosis [2],
             'Прогноз3': prognosis [3],
             'Прогноз4': prognosis [4],
             'Прогноз5': prognosis [5],
             'Прогноз6': prognosis [6],
             'Прогноз7': prognosis [7],
             'Прогноз8': prognosis [8],
             'Прогноз9': prognosis [9],
             'Прогноз10': prognosis [10],
             'Прогноз11': prognosis [11],
             'Прогноз12': prognosis [12],
             'Прогноз13': prognosis [13],
             'Прогноз14': prognosis [14],
             'Прогноз15': prognosis [15] }
    
    return d
    

In [30]:
pdata = pd.DataFrame  ()

for i in new_line(0):
    pdata[i] = None
    

for i in range(data.shape[0]):
    pdata = pdata.append (new_line(i) , ignore_index=True)

In [122]:
its = pd.DataFrame()
d = {'Год':None,
             'Филиал':None,
             'Единица оборудования': None,
             'Класс оборудования':  None,
             'Ошибки': None,  
             'Прогноз0': None,
             'Прогноз1': None,
             'Прогноз2': None,
             'Прогноз3': None,
             'Прогноз4': None,
             'Прогноз5': None,
             'Прогноз6': None,
             'Прогноз7': None,
             'Прогноз8': None,
             'Прогноз9': None,
             'Прогноз10': None,
             'Прогноз11': None,
             'Прогноз12': None,
             'Прогноз13': None,
             'Прогноз14': None,
             'Прогноз15': None }

for i in d:
    its[i] = None
    
for i in unit_dict:
    a = pdata[ (pdata['Год'] == i[0]) & (pdata['Филиал'] == i[1]) & (pdata['Единица оборудования'] == i[2]) ]

    
#-------------------------- Check Data consistency  --------------------
    a4 = ''
    a7 = a['Класс оборудования'].iloc[0]
    
    if a.apply(lambda x: x.nunique())['Класс оборудования']  != 1:
        a4 += 'Разные классы оборудования! '
        for k in a['Класс оборудования']:
            if a7.count(k) != 1:
                a7 += str(k)

  
    unit_cr = dict()
    
    for j in crits_for_unit(a['Класс оборудования'].iloc[0]):
        unit_cr[j] = 0
    

    for j in a['Критерий']:
        unit_cr[j] +=1
    
    for j in unit_cr:
        if (j != 'Состояние в процессе эксплуатации') & (unit_cr[j] != 1):
            a4 += str(j) + ' '  + str(unit_cr[j]) + '! '
    
        if (j == 'Состояние в процессе эксплуатации') & (unit_cr[j] != 2):
            a4 += str(j) + ' ' + str(unit_cr[j]) + '! '

#-------------------------- Check Data consistency  --------------------
    
    
    p = a.sum(axis = 0, numeric_only = True)
    
    nl = {'Год':i[0],
             'Филиал': i[1],
             'Единица оборудования': i[2],
             'Класс оборудования': a7,
             'Ошибки': a4,
             'Прогноз0': p[0],
             'Прогноз1': p[1],
             'Прогноз2': p[2],
             'Прогноз3': p[3],
             'Прогноз4': p[4],
             'Прогноз5': p[5],
             'Прогноз6': p[6],
             'Прогноз7': p[7],
             'Прогноз8': p[8],
             'Прогноз9': p[9],
             'Прогноз10': p[10],
             'Прогноз11': p[11],
             'Прогноз12': p[12],
             'Прогноз13': p[13],
             'Прогноз14': p[14],
             'Прогноз15': p[15] } 
    its = its.append(nl,ignore_index=True)
    

In [123]:
writer = pd.ExcelWriter('output3.xlsx')
#pdata.to_excel(writer,'Units')
its.to_excel(writer,'ITS')
writer.save()
